In [560]:
import numpy as np 
import pandas as pd
from pandas import Series, DataFrame
import json, urllib
from random import randint

In [561]:
books = []
# change search_terms as you like, Google Books API returns at most 40 book records for each search term 
search_terms = ['choose','any','term','you','like'] 
for search_term in search_terms:
    url = "https://www.googleapis.com/books/v1/volumes?q=" + search_term + "+&printType=books&orderBy=newest&maxResults=40"
    response = urllib.urlopen(url)
    data = json.loads(response.read())
    books = np.concatenate([books, data['items']],axis=1)

In [563]:
# np.save("books.npy", books)

In [590]:
# what information to extract from json data is also your choice - below is only my example
# sample json data: https://www.googleapis.com/books/v1/volumes?q=e+&printType=books
cols_bookInfo = ['gbid','title','subtitle','ISBN_13','ISBN_10','publisher','publishedDate','pageCount','description',
             'averageRating','ratingsCount','listPrice']
df_bookInfo = pd.DataFrame(columns=cols_bookInfo)
cols_bookOnShelf = ['bookId','retailPrice','quantity']
df_bookOnShelf = pd.DataFrame(columns=cols_bookOnShelf)
cols_bookAuthor = ['bookId','author','authorOrder']
df_bookAuthor = pd.DataFrame(columns=cols_bookAuthor)
cols_bookCategory = ['bookId','category']
df_bookCategory = pd.DataFrame(columns=cols_bookCategory)

In [591]:
import re
def is_date(s):
    pattern = re.compile("^[0-9]{4}-[0-9]{2}-[0-9]{2}$")
    return pattern.match(s)

In [592]:
bookId = 1
bookAuthorId = 1
bookCategoryId = 1
for book in books:
    gbid = book['id']
    if(gbid in df_bookInfo['gbid'].values):
        continue
    language = book.get('volumeInfo').get('language') 
    if (language != 'en'):
        continue
    listPriceNode = book.get('saleInfo').get('listPrice')
    retailPriceNode = book.get('saleInfo').get('retailPrice')
    if (listPriceNode is None or retailPriceNode is None):
        continue
    listPriceCurrency = listPriceNode.get('currencyCode')
    retailPriceCurrency = retailPriceNode.get('currencyCode')
    if (listPriceCurrency != 'USD' or retailPriceCurrency != 'USD'):
        continue
    publishedDate = book.get('volumeInfo').get('publishedDate')
    if (not is_date(publishedDate)):
        continue
    pageCount = book.get('volumeInfo').get('pageCount')
    if (pageCount is None):
        continue
    ratingsCount = book.get('volumeInfo').get('ratingsCount')
    if (ratingsCount is None):
        continue
    if (book.get('volumeInfo').get('industryIdentifiers') is None):
        continue 
    if (book.get('volumeInfo').get('authors') is None):
        continue
    if (book.get('volumeInfo').get('categories') is None):
        continue
    
    title = book.get('volumeInfo').get('title')
    subtitle = book.get('volumeInfo').get('subtitle')
    for identifierNode in book.get('volumeInfo').get('industryIdentifiers'):
        if (identifierNode.get('type') == 'ISBN_13'):
            isbn13 = identifierNode.get('identifier')
        if (identifierNode.get('type') == 'ISBN_10'):
            isbn10 = identifierNode.get('identifier')    
    publisher = book.get('volumeInfo').get('publisher')
    description = book.get('volumeInfo').get('description')
    averageRating = book.get('volumeInfo').get('averageRating')
    listPrice = book.get('saleInfo').get('listPrice').get('amount')
    retailPrice = book.get('saleInfo').get('retailPrice').get('amount')

    df_bookInfo.loc[bookId] = [gbid,title,subtitle,isbn13,isbn10,publisher,publishedDate,pageCount,description,
                               averageRating,ratingsCount,listPrice]
    df_bookOnShelf.loc[bookId] = [bookId,retailPrice,randint(0,10)]
       
    authorOrder = 1
    for author in book.get('volumeInfo').get('authors'):
        df_bookAuthor.loc[bookAuthorId] = [bookId,author,authorOrder]
        authorOrder = authorOrder + 1
        bookAuthorId = bookAuthorId + 1
    
    for category in book.get('volumeInfo').get('categories'):
        df_bookCategory.loc[bookCategoryId] = [bookId,category.title()]
        bookCategoryId = bookCategoryId + 1
    
    bookId = bookId + 1

In [593]:
df_bookInfo.pageCount = df_bookInfo.pageCount.astype(int)
df_bookInfo.ratingsCount = df_bookInfo.ratingsCount.astype(int)
df_bookInfo.to_csv('bookInfo.csv',sep='\t',encoding='utf-8')
# change encoding in Notepad++ to 'UCS-2 BE BOM', save as 'bookInfo_ucs.csv'
df_bookOnShelf.bookId = df_bookOnShelf.bookId.astype(int)
df_bookOnShelf.quantity = df_bookOnShelf.quantity.astype(int)
df_bookOnShelf.to_csv('bookOnShelf.csv',sep='\t')
df_bookAuthor.bookId = df_bookAuthor.bookId.astype(int)
df_bookAuthor.authorOrder = df_bookAuthor.authorOrder.astype(int)
df_bookAuthor.to_csv('bookAuthor.csv',sep='\t',encoding='utf-8')
# change encoding in Notepad++ to 'UCS-2 BE BOM', save as 'bookAuthor_ucs.csv'
df_bookCategory.bookId = df_bookCategory.bookId.astype(int)
df_bookCategory.to_csv('bookCategory.csv',sep='\t')

In [594]:
# In Microsoft SQL Server 2014

# BULK INSERT dbo.BookInfo
# FROM 'C:\Users\Joy\Documents\IPython\bookInfo_ucs.csv'
# WITH
#(
#    FIRSTROW = 2,
#    FIELDTERMINATOR = '\t',  --CSV field delimiter
#    ROWTERMINATOR = '\n',   --Use to shift the control to next row
#    DATAFILETYPE = 'widechar',
#    TABLOCK
#)

In [597]:
# cover image :
# https://books.google.com/books/content?id=ajLMLYUlAqcC&printsec=frontcover&img=1&zoom=3&edge=curl&source=gbs_api